In [1]:
from __future__  import division
import tarfile
import sklearn as sk
import nltk
import pandas as pd
import numpy as np
import os
from collections import defaultdict
from nltk.translate import AlignedSent
from nltk.translate import Alignment
from nltk.translate import IBMModel
from nltk.translate.ibm_model import Counts
import warnings
from nltk.corpus import comtrans

In [2]:
#Unpacking .tgz files
#tf = tarfile.open("de-en.tgz")
#tf.extractall()
#tf = tarfile.open("fr-en.tgz")
#tf.extractall()

In [3]:
#Loading files
from nltk.corpus import PlaintextCorpusReader
corpus_root = '/home/azick777' 
wordlists = PlaintextCorpusReader(corpus_root, '.*') 
wordlists.fileids()

['.bash_history',
 '.bash_logout',
 '.bash_profile',
 '.bashrc',
 '.cache/abrt/lastnotification',
 '.cache/fontconfig/0251a5afa6ac727a1e32b7d4d4aa7cf0-le64.cache-4',
 '.cache/fontconfig/0b1bcc92b4d25cc154d77dafe3bceaa0-le64.cache-4',
 '.cache/fontconfig/0fef740e1edd47736fa2cccff935ab7c-le64.cache-4',
 '.cache/fontconfig/12513961c6e7090f8648812f9eaf65d6-le64.cache-4',
 '.cache/fontconfig/12b26b760a24f8b4feb03ad48a333a72-le64.cache-4',
 '.cache/fontconfig/211368abcb0ff835c229ff05c9ec01dc-le64.cache-4',
 '.cache/fontconfig/2881ed3fd21ca306ddad6f9b0dd3189f-le64.cache-4',
 '.cache/fontconfig/29c8f5b6bf15d25ebb2e963855ab41be-le64.cache-4',
 '.cache/fontconfig/2e1514a9fdd499050989183bb65136db-le64.cache-4',
 '.cache/fontconfig/3640555adad8a8f6978400293cfce7ab-le64.cache-4',
 '.cache/fontconfig/3830d5c3ddfd5cd38a049b759396e72e-le64.cache-4',
 '.cache/fontconfig/3c3fb04d32a5211b073874b125d29701-le64.cache-4',
 '.cache/fontconfig/3f821257dd33660ba7bbb45c32deb84c-le64.cache-4',
 '.cache/fontconfi

In [4]:
#Reading parallel corpuses
english=wordlists.raw('fr-en.en')
english

'Resumption of the session\nI declare resumed the session of the European Parliament adjourned on Friday 17 December 1999 , and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period .\nAlthough , as you will have seen , the dreaded \' millennium bug \' failed to materialise , still the people in a number of countries suffered a series of natural disasters that truly were dreadful .\nYou have requested a debate on this subject in the course of the next few days , during this part-session .\nIn the meantime , I should like to observe a minute \' s silence , as a number of Members have requested , on behalf of all the victims concerned , particularly those of the terrible storms , in the various countries of the European Union .\nPlease rise , then , for this minute \' s silence .\n( The House rose and observed a minute \' s silence )\nMadam President , on a point of order .\nYou will be aware from the press and television that there h

In [5]:
#reading Parallel corpuses
french=wordlists.raw('fr-en.fr')
french

'Reprise de la session\nJe déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances .\nComme vous avez pu le constater , le grand " bogue de l\' an 2000 " ne s\' est pas produit . En revanche , les citoyens d\' un certain nombre de nos pays ont été victimes de catastrophes naturelles qui ont vraiment été terribles .\nVous avez souhaité un débat à ce sujet dans les prochains jours , au cours de cette période de session .\nEn attendant , je souhaiterais , comme un certain nombre de collègues me l\' ont demandé , que nous observions une minute de silence pour toutes les victimes , des tempêtes notamment , dans les différents pays de l\' Union européenne qui ont été touchés .\nJe vous invite à vous lever pour cette minute de silence .\n( Le Parlement , debout , observe une minute de silence )\nMadame la Présidente , c\' est une motion de procédure .\nVous 

In [6]:
print ('Length of English', len(english))
print ('Length of French', len(french))

Length of English 158486
Length of French 174519


In [7]:
#removing all punctuation and escape characters. Here we should be smart and save apostrophe as it is important
import re
e= english
e = re.sub("[^a-zA-Z'\n' ]+", '',e)
print(e)

Resumption of the session
I declare resumed the session of the European Parliament adjourned on Friday  December   and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period 
Although  as you will have seen  the dreaded ' millennium bug ' failed to materialise  still the people in a number of countries suffered a series of natural disasters that truly were dreadful 
You have requested a debate on this subject in the course of the next few days  during this partsession 
In the meantime  I should like to observe a minute ' s silence  as a number of Members have requested  on behalf of all the victims concerned  particularly those of the terrible storms  in the various countries of the European Union 
Please rise  then  for this minute ' s silence 
 The House rose and observed a minute ' s silence 
Madam President  on a point of order 
You will be aware from the press and television that there have been a number of bomb explosions and k

In [8]:
#The same is done for french 
f= french
f = re.sub("[^a-zA-Z'\n' ]+", '',f)
print(f)

Reprise de la session
Je dclare reprise la session du Parlement europen qui avait t interrompue le vendredi  dcembre dernier et je vous renouvelle tous mes vux en esprant que vous avez pass de bonnes vacances 
Comme vous avez pu le constater  le grand  bogue de l' an   ne s' est pas produit  En revanche  les citoyens d' un certain nombre de nos pays ont t victimes de catastrophes naturelles qui ont vraiment t terribles 
Vous avez souhait un dbat  ce sujet dans les prochains jours  au cours de cette priode de session 
En attendant  je souhaiterais  comme un certain nombre de collgues me l' ont demand  que nous observions une minute de silence pour toutes les victimes  des temptes notamment  dans les diffrents pays de l' Union europenne qui ont t touchs 
Je vous invite  vous lever pour cette minute de silence 
 Le Parlement  debout  observe une minute de silence 
Madame la Prsidente  c' est une motion de procdure 
Vous avez probablement appris par la presse et par la tlvision que plusieu

In [9]:
english_pre = nltk.word_tokenize(e)
french_pre = nltk.word_tokenize(f)

### Problem 1

Here we are trying to get translation Probabilites using the function below

In [10]:
def EM_training( alignSents, num_iter):
        """
        Return the translation probability model trained by EM. 
        Arguments:
        alignSents   -- A list of instances of AlignedSent class, which 
                        contains sentence pairs. 
        num_iter     -- The number of iterations.
        Returns:
        t_ef         -- A dictionary of translation probabilities. 
        """

        # Vocabulary of each language
        fr_vocab = set()
        en_vocab = set()
        for alignSent in alignSents:
            en_vocab.update(alignSent.words)
            fr_vocab.update(alignSent.mots)
        # Add the Null token
        fr_vocab.add(None)

        # Initial probability
        init_prob = 1 / len(en_vocab)

        # Create the translation model with initial probability
        t_ef = defaultdict(lambda: defaultdict(lambda: init_prob))

        total_e = defaultdict(lambda: 0.0)

        for i in range(0, num_iter):
            count_ef = defaultdict(lambda: defaultdict(lambda: 0.0))
            total_f = defaultdict(lambda: 0.0)

            for alignSent in alignSents:
                en_set = alignSent.words
                fr_set = [None] + alignSent.mots  

                # Compute normalization
                for e in en_set:
                    total_e[e] = 0.0
                    for f in fr_set:
                        total_e[e] += t_ef[e][f]

                # Collect counts
                for e in en_set:
                    for f in fr_set:
                        c = t_ef[e][f] / total_e[e]
                        count_ef[e][f] += c
                        total_f[f] += c

            # Compute the estimate probabilities
            for f in fr_vocab:
                for e in en_vocab:
                    t_ef[e][f] = count_ef[e][f] / total_f[f]

        return t_ef     

In [11]:
#Combining as pair of sentences
bitext = []
bitext.append(AlignedSent(english_pre[:3000], french_pre[:3000]))

In [136]:
Trans_prob=EM_training( bitext, 1)
Trans_prob=pd.DataFrame(Trans_prob)
Trans_prob

,','s,A,ABC,ADR,According,Adoption,Affairs,After,Agenda,...,yesterday,yet,you,young,your,yourself,youth,zero,zone,zoning
NaN,0.003674,0.00058,0.000619,0.000039,0.000039,0.000116,0.000039,0.000425,0.000039,0.000039,...,0.000271,0.000309,0.003288,0.000193,0.001199,0.000039,0.000039,0.000077,0.000039,0.000155
Reprise,0.003674,0.00058,0.000619,0.000039,0.000039,0.000116,0.000039,0.000425,0.000039,0.000039,...,0.000271,0.000309,0.003288,0.000193,0.001199,0.000039,0.000039,0.000077,0.000039,0.000155
de,0.003674,0.00058,0.000619,0.000039,0.000039,0.000116,0.000039,0.000425,0.000039,0.000039,...,0.000271,0.000309,0.003288,0.000193,0.001199,0.000039,0.000039,0.000077,0.000039,0.000155
la,0.003674,0.00058,0.000619,0.000039,0.000039,0.000116,0.000039,0.000425,0.000039,0.000039,...,0.000271,0.000309,0.003288,0.000193,0.001199,0.000039,0.000039,0.000077,0.000039,0.000155
session,0.003674,0.00058,0.000619,0.000039,0.000039,0.000116,0.000039,0.000425,0.000039,0.000039,...,0.000271,0.000309,0.003288,0.000193,0.001199,0.000039,0.000039,0.000077,0.000039,0.000155
Je,0.003674,0.00058,0.000619,0.000039,0.000039,0.000116,0.000039,0.000425,0.000039,0.000039,...,0.000271,0.000309,0.003288,0.000193,0.001199,0.000039,0.000039,0.000077,0.000039,0.000155
dclare,0.003674,0.00058,0.000619,0.000039,0.000039,0.000116,0.000039,0.000425,0.000039,0.000039,...,0.000271,0.000309,0.003288,0.000193,0.001199,0.000039,0.000039,0.000077,0.000039,0.000155
reprise,0.003674,0.00058,0.000619,0.000039,0.000039,0.000116,0.000039,0.000425,0.000039,0.000039,...,0.000271,0.000309,0.003288,0.000193,0.001199,0.000039,0.000039,0.000077,0.000039,0.000155
du,0.003674,0.00058,0.000619,0.000039,0.000039,0.000116,0.000039,0.000425,0.000039,0.000039,...,0.000271,0.000309,0.003288,0.000193,0.001199,0.000039,0.000039,0.000077,0.000039,0.000155
Parlement,0.003674,0.00058,0.000619,0.000039,0.000039,0.000116,0.000039,0.000425,0.000039,0.000039,...,0.000271,0.000309,0.003288,0.000193,0.001199,0.000039,0.000039,0.000077,0.000039,0.000155


 #### Problem 2
 
 This class implements the algorithm of Expectation Maximization for 
    the IBM Model 1. 
    
    Step 1 - Run a number of iterations of IBM Model 1 and get the initial
             distribution of translation probability. 
             
    Step 2 - Collect the evidence of a English word being translated by a 
             foreign language word.
             
    Step 3 - Estimate the probability of translation and alignment according 
             the evidence from Step 2. 
             
  

In [12]:
def EM_training_ibm1( alignSents, num_iter):
        """
        Return the translation and alignment probability distributions
        trained by the Expectation Maximization algorithm for IBM Model 1. 
        Arguments:
        alignSents   -- A list contains some sentence pairs. 
        num_iter     -- The number of iterations.
        Returns:
        t_ef         -- A distribution of translation probabilities.
        align        -- A distribution of alignment probabilities.
        
      
        
        
        """

        # Get initial translation probability distribution
        # from a few iterations of Model 1 training.
        t_ef =EM_training(alignSents, 1)

        # Vocabulary of each language
        fr_vocab = set()
        en_vocab = set()
        for alignSent in alignSents:
            en_vocab.update(alignSent.words)
            fr_vocab.update(alignSent.mots)
        fr_vocab.add(None)

        align = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: float))))

        # Initialize the distribution of alignment probability,
        # a(i|j,l_e, l_f) = 1/(l_f + 1)
        for alignSent in alignSents:
            en_set = alignSent.words
            fr_set = [None] + alignSent.mots
            l_f = len(fr_set) - 1
            l_e = len(en_set)
            initial_value = 1 / (l_f + 1)
            for i in range(0, l_f+1):
                for j in range(1, l_e+1):
                    align[i][j][l_e][l_f] = initial_value


        for i in range(0, num_iter):
            count_ef = defaultdict(lambda: defaultdict(float))
            total_f = defaultdict(float)

            count_align = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0.0))))
            total_align = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0.0)))

            total_e = defaultdict(float)

            # for fr_set, en_set in bitexts:
            for alignSent in alignSents:
                en_set = alignSent.words
                fr_set = [None] + alignSent.mots
                l_f = len(fr_set) - 1
                l_e = len(en_set)

                # compute normalization
                for j in range(1, l_e+1):
                    en_word = en_set[j-1]
                    total_e[en_word] = 0
                    for i in range(0, l_f+1):
                        total_e[en_word] += t_ef[en_word][fr_set[i]] * align[i][j][l_e][l_f]

                # collect counts
                for j in range(1, l_e+1):
                    en_word = en_set[j-1]
                    for i in range(0, l_f+1):
                        fr_word = fr_set[i]
                        c = t_ef[en_word][fr_word] * align[i][j][l_e][l_f] / total_e[en_word]
                        count_ef[en_word][fr_word] += c
                        total_f[fr_word] += c
                        count_align[i][j][l_e][l_f] += c
                        total_align[j][l_e][l_f] += c

            # estimate probabilities
            t_ef = defaultdict(lambda: defaultdict(lambda: 0.0))
            align = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0.0))))

            # Smoothing the counts for alignments
            for alignSent in alignSents:
                en_set = alignSent.words
                fr_set = [None] + alignSent.mots
                l_f = len(fr_set) - 1
                l_e = len(en_set)

                laplace = 1.0
                for i in range(0, l_f+1):
                    for j in range(1, l_e+1):
                        value = count_align[i][j][l_e][l_f]
                        if value > 0 and value < laplace:
                            laplace = value

                laplace *= 0.5 
                for i in range(0, l_f+1):
                    for j in range(1, l_e+1):
                        # print i,j,l_e,l_f
                        count_align[i][j][l_e][l_f] += laplace

                initial_value = laplace * l_e
                for j in range(1, l_e+1):
                    total_align[j][l_e][l_f] += initial_value
            
            # Estimate the new lexical translation probabilities
            for f in fr_vocab:
                for e in en_vocab:
                    t_ef[e][f] = count_ef[e][f] / total_f[f]

            # Estimate the new alignment probabilities
            for alignSent in alignSents:
                en_set = alignSent.words
                fr_set = [None] + alignSent.mots
                l_f = len(fr_set) - 1
                l_e = len(en_set)
                for i in range(0, l_f+1):
                    for j in range(1, l_e+1):
                        align[i][j][l_e][l_f] = count_align[i][j][l_e][l_f] / total_align[j][l_e][l_f]

        return t_ef, align

In [14]:
probability, alignmnet_prob=EM_training_ibm1( bitext, 1)

In [15]:
probability=pd.DataFrame(probability)
probability

,','s,ABC,According,Agenda,Alexander,All,Alternatively,Although,America,...,working,would,write,wrong,year,yearold,years,you,young,your
',0.005667,0.001667,0.000333,0.000333,0.000333,0.000667,0.000333,0.000333,0.000667,0.000333,...,0.000333,0.009667,0.000333,0.000333,0.001333,0.000333,0.001,0.007667,0.000333,0.002333
A,0.005667,0.001667,0.000333,0.000333,0.000333,0.000667,0.000333,0.000333,0.000667,0.000333,...,0.000333,0.009667,0.000333,0.000333,0.001333,0.000333,0.001,0.007667,0.000333,0.002333
ABC,0.005667,0.001667,0.000333,0.000333,0.000333,0.000667,0.000333,0.000333,0.000667,0.000333,...,0.000333,0.009667,0.000333,0.000333,0.001333,0.000333,0.001,0.007667,0.000333,0.002333
Alexandre,0.005667,0.001667,0.000333,0.000333,0.000333,0.000667,0.000333,0.000333,0.000667,0.000333,...,0.000333,0.009667,0.000333,0.000333,0.001333,0.000333,0.001,0.007667,0.000333,0.002333
Applaudissements,0.005667,0.001667,0.000333,0.000333,0.000333,0.000667,0.000333,0.000333,0.000667,0.000333,...,0.000333,0.009667,0.000333,0.000333,0.001333,0.000333,0.001,0.007667,0.000333,0.002333
Barents,0.005667,0.001667,0.000333,0.000333,0.000333,0.000667,0.000333,0.000333,0.000667,0.000333,...,0.000333,0.009667,0.000333,0.000333,0.001333,0.000333,0.001,0.007667,0.000333,0.002333
Bush,0.005667,0.001667,0.000333,0.000333,0.000333,0.000667,0.000333,0.000333,0.000667,0.000333,...,0.000333,0.009667,0.000333,0.000333,0.001333,0.000333,0.001,0.007667,0.000333,0.002333
C,0.005667,0.001667,0.000333,0.000333,0.000333,0.000667,0.000333,0.000333,0.000667,0.000333,...,0.000333,0.009667,0.000333,0.000333,0.001333,0.000333,0.001,0.007667,0.000333,0.002333
Comment,0.005667,0.001667,0.000333,0.000333,0.000333,0.000667,0.000333,0.000333,0.000667,0.000333,...,0.000333,0.009667,0.000333,0.000333,0.001333,0.000333,0.001,0.007667,0.000333,0.002333
Communaut,0.005667,0.001667,0.000333,0.000333,0.000333,0.000667,0.000333,0.000333,0.000667,0.000333,...,0.000333,0.009667,0.000333,0.000333,0.001333,0.000333,0.001,0.007667,0.000333,0.002333


In [17]:
alignmnet_prob=pd.DataFrame(alignmnet_prob)
alignmnet_prob

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
1,{3000: {3000: 0.00033325927571669506}},{3000: {3000: 0.00033325927571669506}},{3000: {3000: 0.00033325927571673246}},{3000: {3000: 0.00033325927571693813}},{3000: {3000: 0.0003332592757166952}},{3000: {3000: 0.0003332592757166952}},{3000: {3000: 0.00033325927571669506}},{3000: {3000: 0.00033325927571669506}},{3000: {3000: 0.00033325927571693813}},{3000: {3000: 0.0003332592757166952}},...,{3000: {3000: 0.0003332592757166954}},{3000: {3000: 0.00033325927571669506}},{3000: {3000: 0.0003332592757163757}},{3000: {3000: 0.00033325927571525496}},{3000: {3000: 0.0003332592757166952}},{3000: {3000: 0.0003332592757166954}},{3000: {3000: 0.00033325927571693813}},{3000: {3000: 0.0003332592757166954}},{3000: {3000: 0.0003332592757166952}},{3000: {3000: 0.00033325927571601136}}
2,{3000: {3000: 0.000333259275716693}},{3000: {3000: 0.000333259275716693}},{3000: {3000: 0.00033325927571672726}},{3000: {3000: 0.0003332592757169334}},{3000: {3000: 0.00033325927571669137}},{3000: {3000: 0.00033325927571669094}},{3000: {3000: 0.000333259275716693}},{3000: {3000: 0.000333259275716693}},{3000: {3000: 0.0003332592757169334}},{3000: {3000: 0.00033325927571669137}},...,{3000: {3000: 0.00033325927571669066}},{3000: {3000: 0.000333259275716693}},{3000: {3000: 0.0003332592757163713}},{3000: {3000: 0.00033325927571524943}},{3000: {3000: 0.0003332592757166917}},{3000: {3000: 0.00033325927571669066}},{3000: {3000: 0.0003332592757169334}},{3000: {3000: 0.00033325927571669083}},{3000: {3000: 0.00033325927571669126}},{3000: {3000: 0.0003332592757160069}}
3,{3000: {3000: 0.00033325927571669646}},{3000: {3000: 0.00033325927571669646}},{3000: {3000: 0.0003332592757167302}},{3000: {3000: 0.0003332592757169362}},{3000: {3000: 0.00033325927571669375}},{3000: {3000: 0.0003332592757166936}},{3000: {3000: 0.00033325927571669646}},{3000: {3000: 0.00033325927571669646}},{3000: {3000: 0.0003332592757169362}},{3000: {3000: 0.00033325927571669375}},...,{3000: {3000: 0.00033325927571669354}},{3000: {3000: 0.00033325927571669646}},{3000: {3000: 0.0003332592757163741}},{3000: {3000: 0.0003332592757152523}},{3000: {3000: 0.00033325927571669397}},{3000: {3000: 0.00033325927571669354}},{3000: {3000: 0.0003332592757169362}},{3000: {3000: 0.00033325927571669354}},{3000: {3000: 0.00033325927571669375}},{3000: {3000: 0.0003332592757160097}}
4,{3000: {3000: 0.0003332592757166953}},{3000: {3000: 0.0003332592757166953}},{3000: {3000: 0.00033325927571673197}},{3000: {3000: 0.0003332592757169388}},{3000: {3000: 0.0003332592757166955}},{3000: {3000: 0.0003332592757166955}},{3000: {3000: 0.0003332592757166953}},{3000: {3000: 0.0003332592757166953}},{3000: {3000: 0.0003332592757169388}},{3000: {3000: 0.0003332592757166955}},...,{3000: {3000: 0.0003332592757166953}},{3000: {3000: 0.0003332592757166953}},{3000: {3000: 0.0003332592757163758}},{3000: {3000: 0.0003332592757152536}},{3000: {3000: 0.0003332592757166954}},{3000: {3000: 0.0003332592757166953}},{3000: {3000: 0.0003332592757169388}},{3000: {3000: 0.0003332592757166953}},{3000: {3000: 0.0003332592757166955}},{3000: {3000: 0.00033325927571601147}}
5,{3000: {3000: 0.00033325927571669706}},{3000: {3000: 0.00033325927571669706}},{3000: {3000: 0.00033325927571673165}},{3000: {3000: 0.0003332592757169377}},{3000: {3000: 0.00033325927571669625}},{3000: {3000: 0.00033325927571669554}},{3000: {3000: 0.00033325927571669706}},{3000: {3000: 0.00033325927571669706}},{3000: {3000: 0.0003332592757169377}},{3000: {3000: 0.00033325927571669625}},...,{3000: {3000: 0.0003332592757166951}},{3000: {3000: 0.00033325927571669706}},{3000: {3000: 0.0003332592757163756}},{3000: {3000: 0.00033325927571525377}},{3000: {3000: 0.00033325927571669684}},{3000: {3000: 0.0003332592757166951}},{3000: {3000: 0.0003332592757169377}},{3000: {3000: 0.00033325927571669527}},{3000: {3000: 0.00033325927571669603}},{3000: {3000: 0.00033325927571601125}}
6,{3000: {3000: 0.00033325927571669506}},{3000: {300

In [ ]:
import scipy
i=scipy.sparse.csr_matrix(k.values, dtype='uint16')

i

### Problem 3

In [13]:
#Reading parallel corpuses
english=wordlists.raw('de-en.en')


In [14]:
#reading Parallel corpuses
german=wordlists.raw('de-en.de')


In [15]:
#removing all punctuation and escape characters. Here we should be smart and save apostrophe as it is important
import re
e= english
e = re.sub("[^a-zA-Z'\n' ]+", '',e)
print(e)

Resumption of the session
I declare resumed the session of the European Parliament adjourned on Friday  December   and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period 
Although  as you will have seen  the dreaded ' millennium bug ' failed to materialise  still the people in a number of countries suffered a series of natural disasters that truly were dreadful 
You have requested a debate on this subject in the course of the next few days  during this partsession 
In the meantime  I should like to observe a minute ' s silence  as a number of Members have requested  on behalf of all the victims concerned  particularly those of the terrible storms  in the various countries of the European Union 
Please rise  then  for this minute ' s silence 
 The House rose and observed a minute ' s silence 
Madam President  on a point of order 
You will be aware from the press and television that there have been a number of bomb explosions and k

In [16]:
import re
g= german
g = re.sub("[^a-zA-Z'\n' ]+", '',g)
print(g)

Wiederaufnahme der Sitzungsperiode
Ich erklre die am Freitag  dem  Dezember unterbrochene Sitzungsperiode des Europischen Parlaments fr wiederaufgenommen  wnsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe  da Sie schne Ferien hatten 
Wie Sie feststellen konnten  ist der gefrchtete  MilleniumBug  nicht eingetreten  Doch sind Brger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden 
Im Parlament besteht der Wunsch nach einer Aussprache im Verlauf dieser Sitzungsperiode in den nchsten Tagen 
Heute mchte ich Sie bitten  das ist auch der Wunsch einiger Kolleginnen und Kollegen   allen Opfern der Strme  insbesondere in den verschiedenen Lndern der Europischen Union  in einer Schweigeminute zu gedenken 
Ich bitte Sie  sich zu einer Schweigeminute zu erheben 
 Das Parlament erhebt sich zu einer Schweigeminute  
Frau Prsidentin  zur Geschftsordnung 
Wie Sie sicher aus der Presse und dem Fernsehen wissen  gab es in Sri Lanka mehrere Bombenexplosionen mi

In [17]:
english_pre = nltk.word_tokenize(e)
german_pre = nltk.word_tokenize(g)

In [18]:
#Combining as pair of sentences
bitext2 = []
bitext2.append(AlignedSent(english_pre, german_pre))

In [19]:
probability2, alignmnet_prob2=EM_training_ibm1( bitext2, 1)

In [20]:
probability2=pd.DataFrame(probability2)
probability2

,','s,ABC,According,After,Agenda,Alexander,All,Alternatively,Although,...,working,would,write,wrong,year,yearold,years,you,young,your
Angelegenheit,0.004667,0.002,0.000333,0.000333,0.000333,0.000333,0.000667,0.000667,0.000333,0.000667,...,0.000333,0.007,0.000333,0.000333,0.001667,0.000333,0.000667,0.009,0.000333,0.002333
Antragsteller,0.004667,0.002,0.000333,0.000333,0.000333,0.000333,0.000667,0.000667,0.000333,0.000667,...,0.000333,0.007,0.000333,0.000333,0.001667,0.000333,0.000667,0.009,0.000333,0.002333
Botschaft,0.004667,0.002,0.000333,0.000333,0.000333,0.000333,0.000667,0.000667,0.000333,0.000667,...,0.000333,0.007,0.000333,0.000333,0.001667,0.000333,0.000667,0.009,0.000333,0.002333
Deshalb,0.004667,0.002,0.000333,0.000333,0.000333,0.000333,0.000667,0.000667,0.000333,0.000667,...,0.000333,0.007,0.000333,0.000333,0.001667,0.000333,0.000667,0.009,0.000333,0.002333
Dienstag,0.004667,0.002,0.000333,0.000333,0.000333,0.000333,0.000667,0.000667,0.000333,0.000667,...,0.000333,0.007,0.000333,0.000333,0.001667,0.000333,0.000667,0.009,0.000333,0.002333
EVPEDFraktion,0.004667,0.002,0.000333,0.000333,0.000333,0.000333,0.000667,0.000667,0.000333,0.000667,...,0.000333,0.007,0.000333,0.000333,0.001667,0.000333,0.000667,0.009,0.000333,0.002333
Erklrung,0.004667,0.002,0.000333,0.000333,0.000333,0.000333,0.000667,0.000667,0.000333,0.000667,...,0.000333,0.007,0.000333,0.000333,0.001667,0.000333,0.000667,0.009,0.000333,0.002333
Fakten,0.004667,0.002,0.000333,0.000333,0.000333,0.000333,0.000667,0.000667,0.000333,0.000667,...,0.000333,0.007,0.000333,0.000333,0.001667,0.000333,0.000667,0.009,0.000333,0.002333
Ferien,0.004667,0.002,0.000333,0.000333,0.000333,0.000333,0.000667,0.000667,0.000333,0.000667,...,0.000333,0.007,0.000333,0.000333,0.001667,0.000333,0.000667,0.009,0.000333,0.002333
Gelchter,0.004667,0.002,0.000333,0.000333,0.000333,0.000333,0.000667,0.000667,0.000333,0.000667,...,0.000333,0.007,0.000333,0.000333,0.001667,0.000333,0.000667,0.009,0.000333,0.002333


In [21]:
alignmnet_prob2=pd.DataFrame(alignmnet_prob2)
alignmnet_prob2

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
1,{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757171209}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.00033325927571632713}},{3000: {3000: 0.00033325927571654327}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},...,{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.00033325927571654327}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.00033325927571654327}},{3000: {3000: 0.0003332592757165433}}
2,{3000: {3000: 0.00033325927571656647}},{3000: {3000: 0.00033325927571656647}},{3000: {3000: 0.000333259275717144}},{3000: {3000: 0.00033325927571656647}},{3000: {3000: 0.00033325927571656636}},{3000: {3000: 0.00033325927571656647}},{3000: {3000: 0.0003332592757163502}},{3000: {3000: 0.00033325927571656636}},{3000: {3000: 0.00033325927571656647}},{3000: {3000: 0.00033325927571656647}},...,{3000: {3000: 0.00033325927571656647}},{3000: {3000: 0.00033325927571656636}},{3000: {3000: 0.00033325927571656647}},{3000: {3000: 0.00033325927571656647}},{3000: {3000: 0.00033325927571656647}},{3000: {3000: 0.00033325927571656647}},{3000: {3000: 0.00033325927571656647}},{3000: {3000: 0.00033325927571656647}},{3000: {3000: 0.00033325927571656636}},{3000: {3000: 0.00033325927571656647}}
3,{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.0003332592757171423}},{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.0003332592757163486}},{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.00033325927571656473}},...,{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.00033325927571656473}},{3000: {3000: 0.00033325927571656473}}
4,{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757171209}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.00033325927571632713}},{3000: {3000: 0.00033325927571654327}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},...,{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.00033325927571654327}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.00033325927571654327}},{3000: {3000: 0.0003332592757165433}}
5,{3000: {3000: 0.000333259275716565}},{3000: {3000: 0.000333259275716565}},{3000: {3000: 0.0003332592757171426}},{3000: {3000: 0.000333259275716565}},{3000: {3000: 0.00033325927571656495}},{3000: {3000: 0.000333259275716565}},{3000: {3000: 0.0003332592757163487}},{3000: {3000: 0.00033325927571656495}},{3000: {3000: 0.000333259275716565}},{3000: {3000: 0.000333259275716565}},...,{3000: {3000: 0.000333259275716565}},{3000: {3000: 0.00033325927571656495}},{3000: {3000: 0.000333259275716565}},{3000: {3000: 0.000333259275716565}},{3000: {3000: 0.000333259275716565}},{3000: {3000: 0.000333259275716565}},{3000: {3000: 0.000333259275716565}},{3000: {3000: 0.000333259275716565}},{3000: {3000: 0.00033325927571656495}},{3000: {3000: 0.000333259275716565}}
6,{3000: {3000: 0.0003332592757165433}},{3000: {3000: 0.000333